In [1]:
!unzip -q DIS25k.zip

In [2]:
import os
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models, backend as K

print("TensorFlow version:", tf.__version__)
# Parametreler (ihtiyaca göre değiştir)
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 32
EPOCHS = 40
SEED = 42

# Veri dizinini belirt (burayı kendine göre değiştir)
DATA_DIR = '/content/DIS25k'  # örnek: '/content/drive/MyDrive/DIS25K'
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
MASK_DIR   = os.path.join(DATA_DIR, 'groundtruths')  # eğer farklıysa düzelt

TensorFlow version: 2.19.0


In [3]:
# Tüm resim / maske dosyalarını topla (farklı uzantıları destekleyecek şekilde)
image_files = sorted(glob.glob(os.path.join(IMAGES_DIR, '*')))
mask_files  = sorted(glob.glob(os.path.join(MASK_DIR, '*')))

print("Found images:", len(image_files))
print("Found masks :", len(mask_files))

# Eşleştirme: base filename ile map oluştur (uzantı bağımsız)
imgs_map = {os.path.splitext(os.path.basename(p))[0]: p for p in image_files}
msks_map = {os.path.splitext(os.path.basename(p))[0]: p for p in mask_files}

common_keys = sorted(list(set(imgs_map.keys()).intersection(set(msks_map.keys()))))
print("Matched pairs:", len(common_keys))

images = [imgs_map[k] for k in common_keys]
masks  = [msks_map[k] for k in common_keys]

# Küçük bir kontrol
for i in range(3):
    print(images[i], " <-> ", masks[i])


Found images: 6783
Found masks : 6783
Matched pairs: 6783
/content/DIS25k/images/18181.jpg  <->  /content/DIS25k/groundtruths/18181.jpg
/content/DIS25k/images/18182.jpg  <->  /content/DIS25k/groundtruths/18182.jpg
/content/DIS25k/images/18183.jpg  <->  /content/DIS25k/groundtruths/18183.jpg


In [4]:
from sklearn.model_selection import train_test_split

train_images, val_images, train_masks, val_masks = train_test_split(
    images, masks, test_size=0.2, random_state=SEED)

print("Train pairs:", len(train_images))
print("Val   pairs:", len(val_images))


Train pairs: 5426
Val   pairs: 1357


In [5]:
# Fonksiyonlar: okuma, resize, normalize, augment
AUTOTUNE = tf.data.AUTOTUNE

def read_image_mask(img_path, mask_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)   # jpeg/png farketmez
    img.set_shape([None, None, 3])
    img = tf.image.convert_image_dtype(img, tf.float32)  # [0,1]

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_image(mask, channels=1)
    mask.set_shape([None, None, 1])
    mask = tf.image.convert_image_dtype(mask, tf.float32)  # [0,1]

    # Resize
    img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
    mask = tf.image.resize(mask, [IMG_HEIGHT, IMG_WIDTH], method='nearest')

    # Make binary: (beyaz > 0.5 -> 1)
    mask = tf.where(mask > 0.5, 1.0, 0.0)
    return img, mask

def augment(img, mask):
    # Basit augment: yatay flip, küçük parlaklık değişimleri
    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)
    if tf.random.uniform(()) > 0.7:
        img = tf.image.random_brightness(img, 0.1)
    if tf.random.uniform(()) > 0.7:
        img = tf.image.random_contrast(img, 0.9, 1.1)
    return img, mask

def make_dataset(img_paths, mask_paths, batch_size, augment_flag=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((img_paths, mask_paths))
    if shuffle:
        ds = ds.shuffle(buffer_size=2048, seed=SEED)
    ds = ds.map(lambda x,y: tf.py_function(func=lambda a,b: read_image_mask(a,b),
                                          inp=[x,y], Tout=[tf.float32, tf.float32]),
                num_parallel_calls=AUTOTUNE)
    # tf.py_function ile dönüş tipleri list olarak geldiği için shape set edelim:
    def set_shapes(img, mask):
        img.set_shape([IMG_HEIGHT, IMG_WIDTH, 3])
        mask.set_shape([IMG_HEIGHT, IMG_WIDTH, 1])
        return img, mask
    ds = ds.map(set_shapes, num_parallel_calls=AUTOTUNE)

    if augment_flag:
        ds = ds.map(lambda a,b: tf.py_function(func=lambda x,y: augment(x,y),
                                               inp=[a,b], Tout=[tf.float32, tf.float32]),
                    num_parallel_calls=AUTOTUNE)
        # augment sonrası shape set
        ds = ds.map(set_shapes, num_parallel_calls=AUTOTUNE)

    ds = ds.batch(batch_size).prefetch(AUTOTUNE)
    return ds

train_ds = make_dataset(train_images, train_masks, BATCH_SIZE, augment_flag=True, shuffle=True)
val_ds   = make_dataset(val_images, val_masks, BATCH_SIZE, augment_flag=False, shuffle=False)


In [6]:
from tensorflow.keras import Input

def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    return x

def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2,2))(c)
    return c, p

def decoder_block(x, skip, filters):
    x = layers.UpSampling2D((2,2))(x)
    x = layers.Concatenate()([x, skip])
    x = conv_block(x, filters)
    return x

def build_unet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), n_filters=[64,128,256,512]):
    inputs = Input(input_shape)

    # Encoder
    c1, p1 = encoder_block(inputs, n_filters[0])
    c2, p2 = encoder_block(p1, n_filters[1])
    c3, p3 = encoder_block(p2, n_filters[2])

    # Bottleneck
    bn = conv_block(p3, n_filters[3])

    # Decoder
    d3 = decoder_block(bn, c3, n_filters[2])
    d2 = decoder_block(d3, c2, n_filters[1])
    d1 = decoder_block(d2, c1, n_filters[0])

    outputs = layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(d1)
    model = models.Model(inputs, outputs, name='UNet')
    return model

model = build_unet()
model.summary()


Model: "UNet"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256, 256,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 256, 256,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 256,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 256, 256,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 128, 128,  │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]  

 Total params: 7,794,177 (29.73 MB)

 Trainable params: 7,788,545 (29.71 MB)

 Non-trainable params: 5,632 (22.00 KB)

In [7]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou_metric(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(y_true_f) + K.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=dice_loss,
              metrics=['binary_accuracy', dice_coef, iou_metric])


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# Model save path (Drive'a kaydetmek istersen drive yolu ver)
checkpoint_path = "/content/unet_best.h5"

callbacks = [
    ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_dice_coef', mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, verbose=1),
    EarlyStopping(monitor='val_dice_coef', patience=8, mode='max', restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.7236 - dice_coef: 0.2991 - iou_metric: 0.1778 - loss: 0.7009
Epoch 1: val_dice_coef improved from -inf to 0.07854, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 551s 2s/step - binary_accuracy: 0.7239 - dice_coef: 0.2993 - iou_metric: 0.1779 - loss: 0.7007 - val_binary_accuracy: 0.9001 - val_dice_coef: 0.0785 - val_iou_metric: 0.0409 - val_loss: 0.9215 - learning_rate: 1.0000e-04
Epoch 2/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.8381 - dice_coef: 0.4086 - iou_metric: 0.2585 - loss: 0.5914
Epoch 2: val_dice_coef improved from 0.07854 to 0.26063, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - binary_accuracy: 0.8382 - dice_coef: 0.4087 - iou_metric: 0.2586 - loss: 0.5913 - val_binary_accuracy: 0.8794 - val_dice_coef: 0.2606 - val_iou_metric: 0.1509 - val_loss: 0.7401 - learning_rate: 1.0000e-04
Epoch 3/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.8808 - dice_coef: 0.4608 - iou_metric: 0.3022 - loss: 0.5392
Epoch 3: val_dice_coef improved from 0.26063 to 0.31829, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 317s 2s/step - binary_accuracy: 0.8808 - dice_coef: 0.4608 - iou_metric: 0.3023 - loss: 0.5392 - val_binary_accuracy: 0.8989 - val_dice_coef: 0.3183 - val_iou_metric: 0.1915 - val_loss: 0.6814 - learning_rate: 1.0000e-04
Epoch 4/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9005 - dice_coef: 0.5226 - iou_metric: 0.3560 - loss: 0.4774
Epoch 4: val_dice_coef improved from 0.31829 to 0.44247, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 319s 2s/step - binary_accuracy: 0.9006 - dice_coef: 0.5227 - iou_metric: 0.3560 - loss: 0.4773 - val_binary_accuracy: 0.8153 - val_dice_coef: 0.4425 - val_iou_metric: 0.2865 - val_loss: 0.5571 - learning_rate: 1.0000e-04
Epoch 5/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9138 - dice_coef: 0.5675 - iou_metric: 0.3993 - loss: 0.4325
Epoch 5: val_dice_coef improved from 0.44247 to 0.54669, saving model to /content/unet_best.h5


...

170/170 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - binary_accuracy: 0.9771 - dice_coef: 0.8745 - iou_metric: 0.7786 - loss: 0.1255 - val_binary_accuracy: 0.9637 - val_dice_coef: 0.8130 - val_iou_metric: 0.6884 - val_loss: 0.1859 - learning_rate: 1.0000e-04
Epoch 23/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9783 - dice_coef: 0.8818 - iou_metric: 0.7898 - loss: 0.1182
Epoch 23: val_dice_coef did not improve from 0.81300
170/170 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - binary_accuracy: 0.9783 - dice_coef: 0.8818 - iou_metric: 0.7898 - loss: 0.1182 - val_binary_accuracy: 0.9379 - val_dice_coef: 0.7358 - val_iou_metric: 0.5860 - val_loss: 0.2629 - learning_rate: 1.0000e-04
Epoch 24/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9791 - dice_coef: 0.8850 - iou_metric: 0.7954 - loss: 0.1150
Epoch 24: val_dice_coef improved from 0.81300 to 0.83751, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 314s 2s/step - binary_accuracy: 0.9791 - dice_coef: 0.8850 - iou_metric: 0.7954 - loss: 0.1150 - val_binary_accuracy: 0.9705 - val_dice_coef: 0.8375 - val_iou_metric: 0.7233 - val_loss: 0.1616 - learning_rate: 1.0000e-04
Epoch 25/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9819 - dice_coef: 0.8974 - iou_metric: 0.8148 - loss: 0.1026
Epoch 25: val_dice_coef improved from 0.83751 to 0.83946, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - binary_accuracy: 0.9819 - dice_coef: 0.8974 - iou_metric: 0.8148 - loss: 0.1026 - val_binary_accuracy: 0.9708 - val_dice_coef: 0.8395 - val_iou_metric: 0.7262 - val_loss: 0.1599 - learning_rate: 1.0000e-04
Epoch 26/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9824 - dice_coef: 0.9030 - iou_metric: 0.8241 - loss: 0.0970
Epoch 26: val_dice_coef improved from 0.83946 to 0.84112, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - binary_accuracy: 0.9824 - dice_coef: 0.9030 - iou_metric: 0.8241 - loss: 0.0970 - val_binary_accuracy: 0.9692 - val_dice_coef: 0.8411 - val_iou_metric: 0.7288 - val_loss: 0.1581 - learning_rate: 1.0000e-04
Epoch 27/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9829 - dice_coef: 0.9062 - iou_metric: 0.8294 - loss: 0.0938
Epoch 27: val_dice_coef improved from 0.84112 to 0.85299, saving model to /content/unet_best.h5


170/170 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - binary_accuracy: 0.9829 - dice_coef: 0.9062 - iou_metric: 0.8294 - loss: 0.0938 - val_binary_accuracy: 0.9727 - val_dice_coef: 0.8530 - val_iou_metric: 0.7462 - val_loss: 0.1467 - learning_rate: 1.0000e-04
Epoch 28/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9830 - dice_coef: 0.9053 - iou_metric: 0.8280 - loss: 0.0947
Epoch 28: val_dice_coef did not improve from 0.85299
170/170 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - binary_accuracy: 0.9830 - dice_coef: 0.9053 - iou_metric: 0.8280 - loss: 0.0947 - val_binary_accuracy: 0.9684 - val_dice_coef: 0.8173 - val_iou_metric: 0.6943 - val_loss: 0.1811 - learning_rate: 1.0000e-04
Epoch 29/40
170/170 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - binary_accuracy: 0.9845 - dice_coef: 0.9165 - iou_metric: 0.8468 - loss: 0.0835
Epoch 29: val_dice_coef did not improve from 0.85299
170/170 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - binary_accuracy: 0.9845 - dice_coef: 0.9165 - iou_metric: 0.8467 - loss: 0.0835 - 

In [ ]:
# Basit grafikler
import matplotlib.pyplot as plt

def plot_history(h):
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    plt.plot(h.history['loss'], label='train_loss'); plt.plot(h.history['val_loss'], label='val_loss'); plt.legend(); plt.title('Loss')
    plt.subplot(1,3,2)
    plt.plot(h.history['dice_coef'], label='train_dice'); plt.plot(h.history['val_dice_coef'], label='val_dice'); plt.legend(); plt.title('Dice')
    plt.subplot(1,3,3)
    plt.plot(h.history['iou_metric'], label='train_iou'); plt.plot(h.history['val_iou_metric'], label='val_iou'); plt.legend(); plt.title('IoU')
    plt.show()

plot_history(history)


In [ ]:
import random
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

# Modeli yükle (checkpoint)
model = load_model(checkpoint_path, compile=False)  # compile=False ise custom metrikleri tekrar ayarlamak gerekmez, sadece predict için yeterli
# Eğer tekrar değerlendirmek istersen compile et:
model.compile(optimizer='adam', loss=dice_loss, metrics=[dice_coef, iou_metric])

def show_predictions(model, image_paths, mask_paths, n=5):
    idxs = random.sample(range(len(image_paths)), n)
    for i in idxs:
        img_p = image_paths[i]
        m_p   = mask_paths[i]
        img = tf.io.read_file(img_p)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
        img_np = img.numpy()
        pred = model.predict(np.expand_dims(img_np, 0))[0,...,0]
        pred_mask = (pred > 0.5).astype(np.uint8)

        gt = tf.io.read_file(m_p)
        gt = tf.image.decode_image(gt, channels=1)
        gt = tf.image.resize(gt, [IMG_HEIGHT, IMG_WIDTH], method='nearest')
        gt = tf.image.convert_image_dtype(gt, tf.float32).numpy()[...,0]

        plt.figure(figsize=(12,4))
        plt.subplot(1,3,1); plt.imshow(img_np); plt.title('Image'); plt.axis('off')
        plt.subplot(1,3,2); plt.imshow(gt, cmap='gray'); plt.title('Ground Truth'); plt.axis('off')
        plt.subplot(1,3,3); plt.imshow(pred_mask, cmap='gray'); plt.title('Prediction'); plt.axis('off')
        plt.show()

# Örnek göster
show_predictions(model, val_images, val_masks, n=5)
